### 1 - Exploratory Analysis
This section deals with exploring and analyzing the dataset. It will give us a deeper understanding of the dataset by making us familiar with all the rows and columns of the dataset.

In [1]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import warnings
warnings.simplefilter('ignore')
# nltk
import nltk
nltk.download('words')
nltk.download('wordnet')
nltk.download('stopwords')

# spacy
import spacy
from spacy.lang.en import English
import en_core_web_sm

nlp = spacy.load('en')

# Dependencies
import gensim 
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
!pip install pyldavis
import pyLDAvis.gensim #LDA visualization library

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Gensim
import nltk
nltk.download('stopwords')
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
  
import string 
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('wordnet')
import warnings
warnings.simplefilter('ignore')
from itertools import chain

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 1.6MB 8.9MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=5a40e6bef2ad8b0a9a73960504be858cdab58f0ce16ae514190f03813131bd6d
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet i

In [3]:
# Import Necessary libraries
import os 
# Set Directory path for Dataset for Assia
os.chdir("/content/drive/My Drive/Covid19-Newspaper-Topic-Modeling")

## Fahim preprocessing 

In [4]:
print("Loading Data...")
df_Text = pd.read_csv('Text2.csv')

Loading Data...


In [5]:
df_Text.shape

(15565, 2)

In [7]:
def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", str(elem)))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", str(elem)))
    
    return df

In [8]:
data_clean = clean_text(df_Text, 'Text_en', 'Text_en_clean')

In [9]:
data_clean

,Text_en,Text_bn,Text_en_clean
0,Highest number of Covid-19 deaths in a day. Ni...,একদিনে কোভিড -১৯ এর সর্বোচ্চ সংখ্যা,highest number of covid deaths in a day ninete...
1,"Thrown out of job, Tangail’s chamcham makers p...","চাকরি থেকে ছিটকে পড়ে, টাঙ্গাইলের চামচাম নির্ম...",thrown out of job tangails chamcham makers pas...
2,Coping with Covid-19 Shutdown: 200 families re...,কোভিড -১৯ শাটডাউন মোকাবেলা: ২০০ পরিবার উদ্যানে...,coping with covid shutdown families reaping b...
3,"7 family members, 4 police personnel test posi...","বগুড়ার কোভিড -১৯ এর জন্য সাত পরিবারের সদস্য, ...",family members police personnel test positiv...
4,Coronavirus: Health ministry issues 13 directi...,করোনাভাইরাস: স্বাস্থ্য মন্ত্রনালয় সরকারি কর্ম...,coronavirus health ministry issues directives...
...,...,...,...
15560,The number of victims and deaths is increasing...,লাফিয়ে বাড়ছে আক্রান্ত ও মৃতের সংখ্যা| চীনে করো...,the number of victims and deaths is increasing...
15561,India returns passengers in fear of coronaviru...,করোনাভাইরাস আতঙ্কে যাত্রী ফেরত দিয়েছে ভারত| কর...,india returns passengers in fear of coronaviru...
15562,Instructions to bring back the Bangladeshis tr...,চীনে আটকে পড়া বাংলাদেশিদের ফিরিয়ে আনার নির্দেশ...,instructions to bring back the bangladeshis tr...
15563,Coronavirus: Control room opened in the Depart...,করোনাভাইরাস : স্বাস্থ্য অধিদপ্তরে কন্ট্রোল রুম...,coronavirus control room opened in the departm...


In [10]:
# Clean the data
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'add', 'edu', 'use', 'not', 'would', 'say', 'could', 'be', 'know', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come','1','2','3','4','5','6','7','8','9','0'])

exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    cleanText = str(text)
    cleanText = ' '.join([word for word in cleanText.split() if word not in stop_words])
    cleanText = ''.join(ch for ch in cleanText if ch not in exclude)
    cleanText = ' '.join([lemma.lemmatize(word) for word in cleanText.split()])
    cleanText = ' '.join([word for word in cleanText.split() if len(word) > 2])
    
    return cleanText.split()

data_clean['Text_en_clean_2'] = data_clean['Text_en_clean'].apply(clean)


In [11]:
data_clean

,Text_en,Text_bn,Text_en_clean,Text_en_clean_2
0,Highest number of Covid-19 deaths in a day. Ni...,একদিনে কোভিড -১৯ এর সর্বোচ্চ সংখ্যা,highest number of covid deaths in a day ninete...,"[highest, number, covid, death, day, nineteen,..."
1,"Thrown out of job, Tangail’s chamcham makers p...","চাকরি থেকে ছিটকে পড়ে, টাঙ্গাইলের চামচাম নির্ম...",thrown out of job tangails chamcham makers pas...,"[thrown, job, tangails, chamcham, maker, pas, ..."
2,Coping with Covid-19 Shutdown: 200 families re...,কোভিড -১৯ শাটডাউন মোকাবেলা: ২০০ পরিবার উদ্যানে...,coping with covid shutdown families reaping b...,"[coping, covid, shutdown, family, reaping, ben..."
3,"7 family members, 4 police personnel test posi...","বগুড়ার কোভিড -১৯ এর জন্য সাত পরিবারের সদস্য, ...",family members police personnel test positiv...,"[family, member, police, personnel, test, posi..."
4,Coronavirus: Health ministry issues 13 directi...,করোনাভাইরাস: স্বাস্থ্য মন্ত্রনালয় সরকারি কর্ম...,coronavirus health ministry issues directives...,"[coronavirus, health, ministry, issue, directi..."
...,...,...,...,...
15560,The number of victims and deaths is increasing...,লাফিয়ে বাড়ছে আক্রান্ত ও মৃতের সংখ্যা| চীনে করো...,the number of victims and deaths is increasing...,"[number, victim, death, increasing, leap, bound]"
15561,India returns passengers in fear of coronaviru...,করোনাভাইরাস আতঙ্কে যাত্রী ফেরত দিয়েছে ভারত| কর...,india returns passengers in fear of coronaviru...,"[india, return, passenger, fear, coronavirus]"
15562,Instructions to bring back the Bangladeshis tr...,চীনে আটকে পড়া বাংলাদেশিদের ফিরিয়ে আনার নির্দেশ...,instructions to bring back the bangladeshis tr...,"[instruction, bring, back, bangladeshi, trappe..."
15563,Coronavirus: Control room opened in the Depart...,করোনাভাইরাস : স্বাস্থ্য অধিদপ্তরে কন্ট্রোল রুম...,coronavirus control room opened in the departm...,"[coronavirus, control, room, opened, departmen..."


In [12]:
data_words = data_clean['Text_en_clean_2'].tolist()

In [13]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# !python3 -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

In [14]:
data_clean_en = pd.DataFrame(columns=['Text_en','Text_en_clean', 'data_words','data_ready'])

In [15]:
data_clean_en['Text_en'] = data_clean['Text_en'].copy()
data_clean_en['Text_en_clean'] = data_clean['Text_en_clean'].copy()
data_clean_en['data_words'] = data_words.copy()
data_clean_en['data_ready'] = data_ready.copy()
data_clean_en.head()

,Text_en,Text_en_clean,data_words,data_ready
0,Highest number of Covid-19 deaths in a day. Ni...,highest number of covid deaths in a day ninete...,"[highest, number, covid, death, day, nineteen,...","[high, number, death, day, people, high, numbe..."
1,"Thrown out of job, Tangail’s chamcham makers p...",thrown out of job tangails chamcham makers pas...,"[thrown, job, tangails, chamcham, maker, pas, ...","[throw, job, tangail, maker, hard, day, worker..."
2,Coping with Covid-19 Shutdown: 200 families re...,coping with covid shutdown families reaping b...,"[coping, covid, shutdown, family, reaping, ben...","[cope, covid, shutdown, family, reaping, benef..."
3,"7 family members, 4 police personnel test posi...",family members police personnel test positiv...,"[family, member, police, personnel, test, posi...","[family, member, test, positive, bogura, least..."
4,Coronavirus: Health ministry issues 13 directi...,coronavirus health ministry issues directives...,"[coronavirus, health, ministry, issue, directi...","[issue, official, effort, contain, today, issu..."


In [16]:
print("Generating corpus and dictionary...")
# Create Dictionary
dictionary = corpora.Dictionary(data_ready)
# Create Corpus: Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in data_ready]

Generating corpus and dictionary...


In [17]:
import pickle 
pickle.dump(data_ready, open('output_words.pkl', 'wb'))
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [18]:
data_clean_en.to_csv('data_clean_en.csv',index = False)